<a href="https://colab.research.google.com/github/jmsmg/TIL/blob/main/Python/Concurrency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 병행성(Concurrency)

- 병행성, 흐름제어 설명
- 이터레이터(Iterator) : 반복 가능 객체
- 제너레이터(Generator) : 반복 가능한 객체를 생산해내는 역할 
- \_\_iter__, \_\_next__
- 클래스 기반 제너레이터 구현

In [12]:
# Iterator, Generator
# 파이썬 반복 가능한 객체
# collections, text file, list, dict, set, tuple, unpacking, *args...

# 반복 가능한 이유 -> iter(x) 함수 호출
t = 'ABCD'

print(dir(t))

print('------')

# for c in t:
#   print(c)

print('------')

w = iter(t)

print(next(w))
print(next(w))

while True:
  try:
    print(next(w))
  except StopIteration:
    break

print('------')

# 반복형 확인

from collections import abc

# print(dir(t))
print(hasattr(t, '__iter__'))
print(isinstance(t, abc.Iterable))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']
------
------
A
B
C
D
------
True
True
